# Step1: Load Data

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests

# Load data from SpaceX API
url = 'https://api.spacexdata.com/v4/launches'
response = requests.get(url)
data = response.json()
df = pd.json_normalize(data)
df.head(10)

,static_fire_date_utc,static_fire_date_unix,net,window,rocket,success,failures,details,crew,ships,...,links.reddit.media,links.reddit.recovery,links.flickr.small,links.flickr.original,links.presskit,links.webcast,links.youtube_id,links.article,links.wikipedia,fairings
0,2006-03-17T00:00:00.000Z,1.142554e+09,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 33, 'altitude': None, 'reason': 'mer...",Engine failure at 33 seconds and loss of vehicle,[],[],...,None,None,[],[],None,https://www.youtube.com/watch?v=0a_00nJ_Y88,0a_00nJ_Y88,https://www.space.com/2196-spacex-inaugural-fa...,https://en.wikipedia.org/wiki/DemoSat,NaN
1,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 301, 'altitude': 289, 'reason': 'har...",Successful first stage burn and transition to ...,[],[],...,None,None,[],[],None,https://www.youtube.com/watch?v=Lk4zQ2wP-Nc,Lk4zQ2wP-Nc,https://www.space.com/3590-spacex-falcon-1-roc...,https://en.wikipedia.org/wiki/DemoSat,NaN
2,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,False,"[{'time': 140, 'altitude': 35, 'reason': 'resi...",Residual stage 1 thrust led to collision betwe...,[],[],...,None,None,[],[],None,https://www.youtube.com/watch?v=v0w9p3U8860,v0w9p3U8860,http://www.spacex.com/news/2013/02/11/falcon-1...,https://en.wikipedia.org/wiki/Trailblazer_(sat...,NaN
3,2008-09-20T00:00:00.000Z,1.221869e+09,False,0.0,5e9d0d95eda69955f709d1eb,True,[],Ratsat was carried to orbit on the first succe...,[],[],...,None,None,[],[],None,https://www.youtube.com/watch?v=dLQ2tZEH6G0,dLQ2tZEH6G0,https://en.wikipedia.org/wiki/Ratsat,https://en.wikipedia.org/wiki/Ratsat,NaN
4,None,NaN,False,0.0,5e9d0d95eda69955f709d1eb,True,[],None,[],[],...,None,None,[],[],http://www.spacex.com/press/2012/12/19/spacexs...,https://www.youtube.com/watch?v=yTaIDooc8Og,yTaIDooc8Og,http://www.spacex.com/news/2013/02/12/falcon-1...,https://en.wikipedia.org/wiki/RazakSAT,NaN
5,2010-03-13T00:00:00.000Z,1.268438e+09,False,0.0,5e9d0d95eda69973a809d1ec,True,[],None,[],[],...,None,None,[],[],http://forum.nasaspaceflight.com/index.php?act...,https://www.youtube.com/watch?v=nxSxgBKlYws,nxSxgBKlYws,http://www.spacex.com/news/2013/02/12/falcon-9...,https://en.wikipedia.org/wiki/Dragon_Spacecraf...,NaN
6,2010-12-04T00:00:00.000Z,1.291421e+09,False,0.0,5e9d0d95eda69973a809d1ec,True,[],None,[],[5ea6ed2d080df4000697c901],...,None,None,[],[],http://www.spacex.com/files/downloads/cots1-20...,https://www.youtube.com/watch?v=cdLITgWKe_0,cdLITgWKe_0,https://en.wikipedia.org/wiki/SpaceX_COTS_Demo...,https://en.wikipedia.org/wiki/SpaceX_COTS_Demo...,NaN
7,2012-04-30T00:00:00.000Z,1.335744e+09,False,0.0,5e9d0d95eda69973a809d1ec,True,[],"Launch was scrubbed on first attempt, second l...",[],[5ea6ed2d080df4000697c901],...,None,None,[],[],https://www.nasa.gov/pdf/649910main_cots2_pres...,https://www.youtube.com/watch?v=tpQzDbAY7yI,tpQzDbAY7yI,https://en.wikipedia.org/wiki/Dragon_C2%2B,https://en.wikipedia.org/wiki/Dragon_C2%2B,NaN
8,2012-09-29T00:00:00.000Z,1.348877e+09,False,0.0,5e9d0d95eda69973a809d1ec,True,[],"CRS-1 successful, but the secondary payload wa...",[],[5ea6ed2d080df4000697c902],...,None,None,[],[],https://www.nasa.gov/pdf/694166main_SpaceXCRS-...,https://www.youtube.com/watch?v=-Vk3hiV_zXU,-Vk3hiV_zXU,https://www.nasa.gov/mission_pages/station/mai...,https://en.wikipedia.org/wiki/SpaceX_CRS-1,NaN
9,2013-02-25T18:30:00.000Z,1.361817e+09,False,0.0,5e9d0d95eda69973a809d1ec,True,[],Last launch of the original Falcon 9 v1.0 laun...,[],[5ea6ed2d080df4000697c902],...,None,None,[],[],https://www.nasa.gov/sites/default/files/files...,https://www.youtube.com/watch?v=ik0ElKl5kW4,ik0ElKl5kW4,https://en.wikipedia.org/wiki/SpaceX_CRS-2,https://en.wikipedia.org/wiki/SpaceX_CRS-2,NaN


# Step2: Feature Selection and Cleaning

# Display columns for inspection
df.columns.to_list()

In [3]:
# Select a subset of useful features
df_model = df[['success', 'rocket', 'payloads', 'launchpad', 'date_utc', 'cores']]

# Drop rows with missing 'success' values
df_model = df_model[df_model['success'].notna()].copy()

# Encode 'success' to int
df_model['success'] = df_model['success'].astype(int)

# Count payloads as a simple numeric feature
df_model['payload_count'] = df_model['payloads'].apply(lambda x: len(x) if isinstance(x, list) else 0)

# Extract core count feature
df_model['core_count'] = df_model['cores'].apply(lambda x: len(x) if isinstance(x, list) else 0)

# Convert launch date to year
df_model['launch_year'] = pd.to_datetime(df_model['date_utc']).dt.year

# One-hot encode categorical features like rocket and launchpad
df_encoded = pd.get_dummies(df_model[['rocket', 'launchpad']], drop_first=True)

# Final feature DataFrame
X = pd.concat([df_model[['payload_count', 'core_count', 'launch_year']], df_encoded], axis=1)
y = df_model['success']

X.head()

,payload_count,core_count,launch_year,rocket_5e9d0d95eda69973a809d1ec,rocket_5e9d0d95eda69974db09d1ed,launchpad_5e9e4502f509092b78566f87,launchpad_5e9e4502f509094188566f88,launchpad_5e9e4502f5090995de566f86
0,1,1,2006,False,False,False,False,True
1,1,1,2007,False,False,False,False,True
2,2,1,2008,False,False,False,False,True
3,1,1,2008,False,False,False,False,True
4,1,1,2009,False,False,False,False,True


# Step3: Split Data into Training data and Testing Data

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step:4 Model Selection

In [5]:
from sklearn.linear_model import LinearRegression
model = LinearRegression()

# Step:5 Model Training

In [6]:
model.fit(X_train, y_train)

LinearRegression()

# Step6: Prediction on Training Data

In [7]:
y_train_pred = model.predict(X_train)

# Step7: Prediction on Test  Data 

In [8]:
y_test_pred = model.predict(X_test)

# Step8: Calculating Accuracy

In [10]:
from sklearn.metrics import mean_squared_error, r2_score
print(f'Mean Squared Error: {mean_squared_error(y_test, y_test_pred):.4f}')
print(f'R² Score: {r2_score(y_train, y_train_pred):.4f}')

Mean Squared Error: 0.0009
R² Score: 0.3623


In [13]:
import requests
import pandas as pd

def load_and_save_spacex_data(filename="data.csv"):
    """
    Fetches launch data from the SpaceX API and saves it as a CSV file.
    
    Parameters:
        filename (str): The name of the CSV file to save the data to. Default is 'data.csv'.
    """
    url = 'https://api.spacexdata.com/v4/launches'
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        df = pd.json_normalize(data)
        df.to_csv(filename, index=False)
        print(f"Data saved to {filename}")
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")

# Example usage
load_and_save_spacex_data()


Data saved to data.csv
